# Euler Angles Functions

In [1]:
import numpy as np

# 1) Rotation Matrix Defintions

In [2]:
def rotation_matrix_x(phi, transformation_type='passive'):
    """Generate rotation matrix for a roll (rotation about the x-axis).
    
    Args:
        phi (float): The angle of rotation in degrees.
        transformation_type (str): Specifies the type of transformation, 'passive' (default) or 'active'.
    
    Returns:
        numpy.ndarray: The rotation matrix for x-axis rotation.
    """
    phi = np.radians(phi)
    c, s = np.cos(phi), np.sin(phi)
    matrix = np.array([[1,  0,  0], 
                       [0,  c,  s], 
                       [0, -s, c]])
    
    if transformation_type == 'active':
        return matrix.T
    
    return matrix

def rotation_matrix_y(theta, transformation_type='passive'):
    """Generate rotation matrix for a pitch (rotation about the y-axis).
    
    Args:
        theta (float): The angle of rotation in degrees.
        transformation_type (str): Specifies the type of transformation, 'passive' (default) or 'active'.
    
    Returns:
        numpy.ndarray: The rotation matrix for y-axis rotation.
    """
    theta = np.radians(theta)
    c, s = np.cos(theta), np.sin(theta)
    matrix = np.array([[c,  0, -s], 
                       [0,  1,  0], 
                       [s,  0,  c]])
    
    if transformation_type == 'active':
        return matrix.T
    
    return matrix

def rotation_matrix_z(psi, transformation_type='passive'):
    """Generate rotation matrix for a yaw (rotation about the z-axis).
    
    Args:
        psi (float): The angle of rotation in degrees.
        transformation_type (str): Specifies the type of transformation, 'passive' (default) or 'active'.
    
    Returns:
        numpy.ndarray: The rotation matrix for z-axis rotation.
    """
    psi = np.radians(psi)
    c, s = np.cos(psi), np.sin(psi)
    matrix = np.array([[ c,  s,  0], 
                       [-s,  c,  0], 
                       [ 0,  0,  1]])
    
    if transformation_type == 'active':
        return matrix.T
    
    return matrix

# 2) Euler_to_DCM

In [3]:
def Euler_to_DCM(angles, sequence, transformation_type='passive'):
    """
    Converts a set of Euler angles into a Direction Cosine Matrix (DCM)
    based on the specified rotation sequence.

    Args:
        angles (array-like): A list or array of three rotation angles in degrees,
                             corresponding to the axes in the rotation sequence.
        sequence (str): The rotation sequence as a string (e.g., '321' or 'zyx').
                        The sequence defines the order and axes of rotations.
        transformation_type (str): 'active' or 'passive' rotation.

    Returns:
        numpy.ndarray: A 3x3 Direction Cosine Matrix (DCM).
    """
    if len(angles) != 3:
        raise ValueError("Angles must be a list or array of three elements.")

    if len(sequence) != 3:
        raise ValueError("Rotation sequence must be a string of three characters.")

    # Convert the sequence to lower case to handle both upper and lower case inputs
    sequence = sequence.lower()

    # Validate and map the rotation sequence to 'x', 'y', 'z'
    valid_axes = {'1': 'x', 
                  '2': 'y', 
                  '3': 'z', 
                  'x': 'x', 
                  'y': 'y', 
                  'z': 'z'}
    mapped_sequence = ''
    for axis_char in sequence:
        if axis_char not in valid_axes:
            raise ValueError(f"Invalid axis '{axis_char}' in rotation sequence. Use '1', '2', '3', 'x', 'y', or 'z'.")
        mapped_sequence += valid_axes[axis_char]

    # Now mapped_sequence contains only 'x', 'y', 'z'
    # Map axis characters to rotation functions
    axis_functions = {
        'x': rotation_matrix_x,
        'y': rotation_matrix_y,
        'z': rotation_matrix_z
    }

    # List to store the rotation matrices
    rotation_matrices = []

    # Map each angle to its corresponding axis in the mapped sequence
    for angle_deg, axis_char in zip(angles, mapped_sequence):
        # Get the corresponding rotation function
        rotation_function = axis_functions[axis_char]
        
        # Compute the rotation matrix
        R = rotation_function(angle_deg, transformation_type=transformation_type)
        
        # Append to the list
        rotation_matrices.append(R)

    # Initialize the DCM as an identity matrix
    DCM = np.eye(3)

    # Apply the rotations in reverse order for proper sequencing
    # Since rotations are applied right-to-left (R3 * R2 * R1 * v)
    for R in rotation_matrices:
        DCM = np.matmul(R, DCM)

    return DCM

## 2.1 - Functional Testing of Euler_to_DCM

In [4]:
from scipy.spatial.transform import Rotation as R

# List of all 12 possible Euler angle sequences
sequences = ['121', '123', '131', '132', '212', '213',
             '231', '232', '312', '313', '321', '323']

# Input angles
angles_input = [30, 45, 60]    # Angles in degrees
transformation_type = 'active' # scipy rotation matrix is in active conventions

# Mapping from numeric axes to alphabetic axes for SciPy
axis_map = {'1': 'x', 
            '2': 'y', 
            '3': 'z'}

for sequence in sequences:
    # Map numeric sequence to alphabetic sequence for SciPy
    sequence_scipy = ''.join([axis_map[axis] for axis in sequence])
    
    # Generate DCM using your function
    DCM_custom = Euler_to_DCM(angles_input, sequence, transformation_type)
    
    # Generate DCM using SciPy
    r = R.from_euler(sequence_scipy, angles_input, degrees=True)
    DCM_scipy = r.as_matrix()
    
    # Adjust for rotation conventions (passive vs. active)
    # Since your DCM is passive and SciPy's is active, transpose SciPy's DCM
    #DCM_scipy_T = DCM_scipy  # Transpose to match passive convention
    
    # Compare DCMs
    DCM_difference = DCM_custom - DCM_scipy
    max_diff = np.max(np.abs(DCM_difference))
    
    # Print results
    print(f"Sequence      : {sequence}")
    print(f"SciPy Sequence: {sequence_scipy}")
    print("")
    print("DCM from your function:")
    print(DCM_custom)
    print("")
    print("DCM from SciPy:")
    print(DCM_scipy)
    print("")
    print(f"Max difference between custom DCM and SciPy DCM: {max_diff:.2e}")
    print("-" * 100)

Sequence      : 121
SciPy Sequence: xyx

DCM from your function:
[[ 0.70710678  0.35355339  0.61237244]
 [ 0.61237244  0.12682648 -0.78033009]
 [-0.35355339  0.9267767  -0.12682648]]

DCM from SciPy:
[[ 0.70710678  0.35355339  0.61237244]
 [ 0.61237244  0.12682648 -0.78033009]
 [-0.35355339  0.9267767  -0.12682648]]

Max difference between custom DCM and SciPy DCM: 1.11e-16
----------------------------------------------------------------------------------------------------
Sequence      : 123
SciPy Sequence: xyz

DCM from your function:
[[ 0.35355339 -0.5732233   0.73919892]
 [ 0.61237244  0.73919892  0.28033009]
 [-0.70710678  0.35355339  0.61237244]]

DCM from SciPy:
[[ 0.35355339 -0.5732233   0.73919892]
 [ 0.61237244  0.73919892  0.28033009]
 [-0.70710678  0.35355339  0.61237244]]

Max difference between custom DCM and SciPy DCM: 1.11e-16
----------------------------------------------------------------------------------------------------
Sequence      : 131
SciPy Sequence: xzx

DCM

# 3) DCM_to_Euler

In [5]:
def DCM_to_Euler(DCM, sequence, transformation_type='passive'):
    """
    Converts a Direction Cosine Matrix (DCM) into a set of Euler angles
    based on the specified rotation sequence.

    Args:
        DCM (numpy.ndarray): A 3x3 Direction Cosine Matrix.
        sequence (str): The rotation sequence as a string (e.g., '321' or 'zyx').
                        The sequence defines the order and axes of rotations.
        transformation_type (str): 'active' or 'passive' rotation.

    Returns:
        tuple: A tuple of three Euler angles in degrees, corresponding to the axes in the rotation sequence.

    Notes:
        - The function handles both 'active' and 'passive' transformations.
        - The output angles are in degrees.
        - The function accounts for possible singularities in the rotation representation.
    """
    # Validate the input DCM
    if DCM.shape != (3, 3):
        raise ValueError("DCM must be a 3x3 matrix.")

    # Validate and parse the rotation sequence
    if len(sequence) != 3:
        raise ValueError("Rotation sequence must be a string of three characters.")

    # Convert the sequence to lower case to handle both upper and lower case inputs
    sequence = sequence.lower()

    # Validate and map the rotation sequence to '1', '2', '3'
    valid_axes = {'1', '2', '3', 'x', 'y', 'z'}
    axis_map = {'x': '1', 
                'y': '2', 
                'z': '3'}
    seq_mapped = ''
    for axis_char in sequence:
        if axis_char not in valid_axes:
            raise ValueError(f"Invalid axis '{axis_char}' in rotation sequence.")
        seq_mapped += axis_map.get(axis_char, axis_char)

    # Handle the transformation type once
    if transformation_type == 'active':
        DCM = DCM.T  # Transpose for active transformation
    elif transformation_type != 'passive':
        raise ValueError("transformation_type must be 'active' or 'passive'.")

    # Extract angles based on the rotation sequence
    seq = seq_mapped

    # Initialize angles
    theta_1 = 0
    theta_2 = 0
    theta_3 = 0

    # Use if-elif chain to handle different sequences
    if seq == '121':
        theta_1 = np.arctan2(DCM[0,1], -DCM[0,2])
        theta_2 = np.arccos(DCM[0,0])
        theta_3 = np.arctan2(DCM[1,0], DCM[2,0])
    
    elif seq == '123':
        theta_1 = np.arctan2(-DCM[2,1], DCM[2,2])
        theta_2 = np.arcsin(DCM[2,0])
        theta_3 = np.arctan2(-DCM[1,0], DCM[0,0])
    
    elif seq == '131':
        theta_1 = np.arctan2(DCM[0,2], DCM[0,1])
        theta_2 = np.arccos(DCM[0,0])
        theta_3 = np.arctan2(DCM[2,0], -DCM[1,0])
    
    elif seq == '132':
        theta_1 = np.arctan2(DCM[1,2], DCM[1,1])
        theta_2 = np.arcsin(-DCM[1,0])
        theta_3 = np.arctan2(DCM[2,0], DCM[0,0])
    
    elif seq == '212':
        theta_1 = np.arctan2(DCM[1,0], DCM[1,2])
        theta_2 = np.arccos(DCM[1,1])
        theta_3 = np.arctan2(DCM[0,1], -DCM[2,1])
    
    elif seq == '213':
        theta_1 = np.arctan2(DCM[2,0], DCM[2,2])
        theta_2 = np.arcsin(-DCM[2,1])
        theta_3 = np.arctan2(DCM[0,1], DCM[1,1])
    
    elif seq == '231':
        theta_1 = np.arctan2(-DCM[0,2], DCM[0,0])
        theta_2 = np.arcsin(DCM[0,1])
        theta_3 = np.arctan2(-DCM[2,1], DCM[1,1])
    
    elif seq == '232':
        theta_1 = np.arctan2(DCM[1,2], -DCM[1,0])
        theta_2 = np.arccos(DCM[1,1])
        theta_3 = np.arctan2(DCM[2,1], DCM[0,1])
    
    elif seq == '312':
        theta_1 = np.arctan2(-DCM[1,0], DCM[1,1])
        theta_2 = np.arcsin(DCM[1,2])
        theta_3 = np.arctan2(-DCM[0,2], DCM[2,2])
    
    elif seq == '313':
        theta_1 = np.arctan2(DCM[2,0], -DCM[2,1])
        theta_2 = np.arccos(DCM[2,2])
        theta_3 = np.arctan2(DCM[0,2], DCM[1,2])
    
    elif seq == '321':
        theta_1 = np.arctan2(DCM[0,1], DCM[0,0])
        theta_2 = np.arcsin(-DCM[0,2])
        theta_3 = np.arctan2(DCM[1,2], DCM[2,2])
    
    elif seq == '323':
        theta_1 = np.arctan2(DCM[2,1], DCM[2,0])
        theta_2 = np.arccos(DCM[2,2])
        theta_3 = np.arctan2(DCM[1,2], -DCM[0,2])
    
    else:
        raise NotImplementedError(f"Rotation sequence '{sequence}' is not implemented.")

    # Convert angles from radians to degrees
    angles_deg = np.rad2deg([theta_1, theta_2, theta_3])

    return tuple(angles_deg)

## 3.1 - Functional Testing of DCM_to_Euler

In [6]:
# List of all 12 possible Euler angle sequences
sequences = ['121', '123', '131', '132',
             '212', '213', '231', '232',
             '312', '313', '321', '323']

# Input angles
angles_input = [-30, 90, 0]  # Angles in degrees
transformation_type = 'passive'  # Can be 'passive' or 'active' as needed

for sequence in sequences:
    # Generate DCM using your Eulerto_DCM function
    DCM = Euler_to_DCM(angles_input, sequence, transformation_type)
    
    # Extract Euler angles from the DCM using your DCM_to_Euler function
    angles_output = DCM_to_Euler(DCM, sequence, transformation_type)
    
    # Compute the difference between input and output angles
    angle_difference = np.array(angles_input) - np.array(angles_output)
    
    # Print results
    print(f"Sequence      : {sequence}")
    print("")
    print("Input Euler Angles    :", angles_input)
    print("Extracted Euler Angles:", angles_output)
    print("")
    print("Difference between input and extracted angles:", angle_difference)
    print("-" * 100)

Sequence      : 121

Input Euler Angles    : [-30, 90, 0]
Extracted Euler Angles: (-29.999999999999996, 90.0, 0.0)

Difference between input and extracted angles: [-3.55271368e-15  0.00000000e+00  0.00000000e+00]
----------------------------------------------------------------------------------------------------
Sequence      : 123

Input Euler Angles    : [-30, 90, 0]
Extracted Euler Angles: (-29.999999999999996, 90.0, -0.0)

Difference between input and extracted angles: [-3.55271368e-15  0.00000000e+00  0.00000000e+00]
----------------------------------------------------------------------------------------------------
Sequence      : 131

Input Euler Angles    : [-30, 90, 0]
Extracted Euler Angles: (-29.999999999999996, 90.0, 0.0)

Difference between input and extracted angles: [-3.55271368e-15  0.00000000e+00  0.00000000e+00]
----------------------------------------------------------------------------------------------------
Sequence      : 132

Input Euler Angles    : [-30, 90, 0]

# 4) BodyRate_to_EulerRate

In [7]:
def BodyRate_to_EulerRate(angles, omega, sequence):
    """
    Converts body rates (angular velocities in the body frame) to Euler angle rates
    based on the specified rotation sequence and Euler angles.

    Args:
        angles (list or tuple): Euler angles [theta_1, theta_2, theta_3] in degrees.
        omega (list or tuple): Body rates [omega1, omega2, omega3] in radians per second.
        sequence (str): The rotation sequence as a string (e.g., '321', 'ZYX').

    Returns:
        numpy.ndarray: Euler angle rates [theta_1_dot, theta_2_dot, theta_3_dot] in degrees per second.

    Notes:
        - The function computes the B matrix based on the rotation sequence and Euler angles.
        - It handles both proper Euler angles and Tait-Bryan angles.
        - Angles are input and output in degrees; internal computations are in radians.
        - The axis indices correspond to 1-based indexing to match standard notation.
    """
    # Validate input lengths
    if len(angles) != 3:
        raise ValueError("The 'angles' parameter must have three elements.")
    if len(omega) != 3:
        raise ValueError("The 'omega' parameter must have three elements.")
    if len(sequence) != 3:
        raise ValueError("The 'sequence' parameter must be a string of three characters.")

    # Convert input angles from degrees to radians
    angles_rad = np.deg2rad(angles)
    theta_1, theta_2, theta_3 = angles_rad  # Euler angles in radians

    # Convert omega to a NumPy array
    omega = np.array(omega)  # Body rates in radians per second

    # Map axes to indices (1-based indexing for clarity)
    axis_map = {'1': 1, '2': 2, '3': 3,
                'x': 1, 'y': 2, 'z': 3}

    # Convert sequence to indices
    sequence = sequence.lower()
    try:
        axes = [axis_map[axis] for axis in sequence]
    except KeyError as e:
        raise ValueError(f"Invalid axis '{e.args[0]}' in rotation sequence.")

    # Precompute trigonometric functions with subscripts matching theta_1, theta_2, theta_3
    s1 = np.sin(theta_1)
    c1 = np.cos(theta_1)
    s2 = np.sin(theta_2)
    c2 = np.cos(theta_2)
    s3 = np.sin(theta_3)
    c3 = np.cos(theta_3)

    # Initialize the B matrix
    B = np.zeros((3, 3))

    # Define a small threshold to avoid division by zero
    epsilon = np.finfo(float).eps

    # Compute the B matrix based on the rotation sequence
    # The indices in B correspond to the 1-based axis indices minus 1 (for zero-based indexing)
    if sequence == '121':
        if abs(s2) < epsilon:
            raise ValueError("Singularity encountered: sin(theta_2) is zero.")
        B[0, :] = [0, s3, c3]
        B[1, :] = [0, s2 * c3, -s2 * s3]
        B[2, :] = [s2, -c2 * s3, -c2 * c3]
        B /= s2
        
    elif sequence == '123':
        if abs(c2) < epsilon:
            raise ValueError("Singularity encountered: cos(theta_2) is zero.")
        B[0, :] = [c3, -s3, 0]
        B[1, :] = [c2 * s3, c2 * c3, 0]
        B[2, :] = [-s2 * c3, s2 * s3, c2]
        B /= c2
        
    elif sequence == '131':
        if abs(s2) < epsilon:
            raise ValueError("Singularity encountered: sin(theta_2) is zero.")
        B[0, :] = [0, -c3, s3]
        B[1, :] = [0, s2 * s3, s2 * c3]
        B[2, :] = [s2, c2 * c3, -c2 * s3]
        B /= s2
    elif sequence == '132':
        if abs(c2) < epsilon:
            raise ValueError("Singularity encountered: cos(theta_2) is zero.")
        B[0, :] = [c3, 0, s3]
        B[1, :] = [-c2 * s3, 0, c2 * c3]
        B[2, :] = [s2 * c3, c2, s2 * s3]
        B /= c2
        
    elif sequence == '212':
        if abs(s2) < epsilon:
            raise ValueError("Singularity encountered: sin(theta_2) is zero.")
        B[0, :] = [s3, 0, -c3]
        B[1, :] = [s2 * c3, 0, s2 * s3]
        B[2, :] = [-c2 * s3, s2, c2 * c3]
        B /= s2
        
    elif sequence == '213':
        if abs(c2) < epsilon:
            raise ValueError("Singularity encountered: cos(theta_2) is zero.")
        B[0, :] = [s3, c3, 0]
        B[1, :] = [c2 * c3, -c2 * s3, 0]
        B[2, :] = [s2 * s3, s2 * c3, c2]
        B /= c2
        
    elif sequence == '231':
        if abs(c2) < epsilon:
            raise ValueError("Singularity encountered: cos(theta_2) is zero.")
        B[0, :] = [0, c3, -s3]
        B[1, :] = [0, c2 * s3, c2 * c3]
        B[2, :] = [c2, -s2 * c3, s2 * s3]
        B /= c2
    elif sequence == '232':
        if abs(s2) < epsilon:
            raise ValueError("Singularity encountered: sin(theta_2) is zero.")
        B[0, :] = [c3, 0, s3]
        B[1, :] = [-s2 * s3, 0, s2 * c3]
        B[2, :] = [-c2 * c3, s2, -c2 * s3]
        B /= s2
        
    elif sequence == '312':
        if abs(c2) < epsilon:
            raise ValueError("Singularity encountered: cos(theta_2) is zero.")
        B[0, :] = [-s3, 0, c3]
        B[1, :] = [c2 * c3, 0, c2 * s3]
        B[2, :] = [s2 * s3, c2, -s2 * c3]
        B /= c2
    elif sequence == '313':
        if abs(s2) < epsilon:
            raise ValueError("Singularity encountered: sin(theta_2) is zero.")
        B[0, :] = [s3, c3, 0]
        B[1, :] = [c3 * s2, -s3 * s2, 0]
        B[2, :] = [-s3 * c2, -c3 * c2, s2]
        B /= s2
        
    elif sequence == '321':
        if abs(c2) < epsilon:
            raise ValueError("Singularity encountered: cos(theta_2) is zero.")
        B[0, :] = [0, s3, c3]
        B[1, :] = [0, c2 * c3, -c2 * s3]
        B[2, :] = [c2, s2 * s3, s2 * c3]
        B /= c2
        
    elif sequence == '323':
        if abs(s2) < epsilon:
            raise ValueError("Singularity encountered: sin(theta_2) is zero.")
        B[0, :] = [-c3, s3, 0]
        B[1, :] = [s2 * s3, s2 * c3, 0]
        B[2, :] = [c2 * c3, -c2 * s3, s2]
        B /= s2
        
    else:
        raise NotImplementedError(f"Rotation sequence '{sequence}' is not implemented.")

    # Compute Euler angle rates in radians per second
    euler_rates_rad = np.matmul(B, omega)

    # Convert Euler rates from radians per second to degrees per second
    euler_rates_deg = np.rad2deg(euler_rates_rad)

    return euler_rates_deg

In [8]:
# Example usage of the updated BodyRate_to_EulerRate function
angles = [30, 90, 60]  # q1, q2, q3 in degrees
omega = [0.1, 0.2, 0.3]  # omega1, omega2, omega3 in radians per second
sequence = '123'  # Rotation sequence

# Compute Euler angle rates
euler_rates = BodyRate_to_EulerRate(angles, omega, sequence)

print("Euler angle rates (degrees per second):", euler_rates)

ValueError: Singularity encountered: cos(theta_2) is zero.

# 5) EulerRate_to_BodyRate